# Routing Engine Prototype

Python implementation of CH routing algorithms for testing and comparison.

## Algorithms
1. **Classic**: Bidirectional Dijkstra with `inside` filtering
2. **Pruned**: + H3 `parent_check` on popped nodes
3. **Many-to-Many**: Multi-source/target for KNN

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
#import h3.api.basic_int as h3  # Use integer API for H3
import h3
from collections import defaultdict
from heapq import heappush, heappop
from dataclasses import dataclass
from typing import Optional
import time

## Configuration

In [2]:
# Data paths
SHORTCUTS_PATH = "/home/kaveh/projects/shortcuts-generation/output/Somerset_shortcuts"
EDGES_PATH = "/home/kaveh/projects/shortcuts-generation/data/Somerset_driving_simplified_edges_with_h3.csv"

## Data Structures

In [3]:
@dataclass
class QueryResult:
    distance: float
    path: list
    reachable: bool

@dataclass
class HighCell:
    cell: int
    res: int

@dataclass
class Shortcut:
    from_edge: int
    to_edge: int
    cost: float
    via_edge: int
    cell: int
    inside: int

## Load Data

In [4]:
# Load shortcuts with explicit int64 for cell column
shortcuts_df = pq.read_table(SHORTCUTS_PATH).to_pandas()
# CRITICAL: Convert cell to int64 immediately to avoid float precision loss
shortcuts_df['cell'] = pd.to_numeric(shortcuts_df['cell'], errors='coerce').fillna(0).astype('int64')
print(f"Loaded {len(shortcuts_df):,} shortcuts")
print(f"Cell dtype: {shortcuts_df['cell'].dtype}")
print(shortcuts_df.head())

Loaded 378,141 shortcuts
Cell dtype: int64
   incoming_edge  outgoing_edge       cost  via_edge  inside  \
0              0             88  46.250016       892      -2   
1              0            109  43.006350       892      -2   
2              0            648   6.292683       933       1   
3              0            920   4.828367       926      -1   
4              0           1529   9.138313         2       0   

                 cell  
0  613699779993534463  
1  613699779993534463  
2                   0  
3  613699779993534463  
4  613699779993534463  


In [5]:
# Load edge metadata
edges_df = pd.read_csv(EDGES_PATH)
print(f"Loaded {len(edges_df):,} edges")
print(edges_df.head())

Loaded 5,900 edges
      source     target   length  maxspeed  \
0  167790704  167862530   80.287      60.0   
1  167790704  167790717  102.097      60.0   
2  167790704  167825885  323.104      50.0   
3  167790717  167790719  111.432      60.0   
4  167790717  167862553   84.317      30.0   

                                            geometry      highway       cost  \
0  LINESTRING (-84.60797882080078 37.091793060302...    secondary   4.817220   
1  LINESTRING (-84.60797882080078 37.091793060302...    secondary   6.125820   
2  LINESTRING (-84.60797882080078 37.091793060302...     tertiary  23.263488   
3  LINESTRING (-84.60697937011719 37.091552734375...    secondary   6.685920   
4  LINESTRING (-84.60697937011719 37.091552734375...  residential  10.118040   

        incoming_cell       outgoing_cell  lca_res  id  
0  645224977384028320  645224977383611141        8   0  
1  645224977383614665  645224977383611141       10   1  
2  645224977384658840  645224977383611141        8  

In [6]:
# Build edge metadata lookup
edge_meta = {}
for _, row in edges_df.iterrows():
    edge_meta[row['id']] = {
        'incoming_cell': int(row['incoming_cell']),
        'outgoing_cell': int(row['outgoing_cell']),
        'lca_res': int(row['lca_res']),
        'length': float(row['length']),
        'cost': float(row['cost'])
    }
print(f"Built metadata for {len(edge_meta):,} edges")

Built metadata for 5,900 edges


In [7]:
# Build adjacency lists
fwd_adj = defaultdict(list)
bwd_adj = defaultdict(list)
# Convert cell column to int64 first to avoid float precision loss
# Build adjacency lists from shortcuts
fwd_adj = defaultdict(list)
bwd_adj = defaultdict(list)
for row in shortcuts_df.itertuples():
    sc = Shortcut(
        from_edge=row.incoming_edge,
        to_edge=row.outgoing_edge,
        cost=row.cost,
        via_edge=row.via_edge,
        inside=row.inside,
        cell=row.cell
    )
    fwd_adj[sc.from_edge].append(sc)
    bwd_adj[sc.to_edge].append(sc)
print(f"Built adjacency: {len(fwd_adj)} forward, {len(bwd_adj)} backward")

Built adjacency: 5895 forward, 5898 backward


## H3 Utility Functions

In [8]:
import h3  # Use the standard h3 import (not h3.api.basic_int)
def safe_cell_to_parent(cell: int, target_res: int) -> int:
    """Get parent cell at target resolution, handling edge cases."""
    if cell == 0 or target_res < 0:
        return 0
    cell_str = h3.int_to_str(cell)
    cell_res = h3.get_resolution(cell_str)
    if target_res > cell_res:
        return cell
    return h3.str_to_int(h3.cell_to_parent(cell_str, target_res))
    
def find_lca(cell1: int, cell2: int) -> int:
    """Find lowest common ancestor of two H3 cells."""
    if cell1 == 0 or cell2 == 0:
        return 0
    cell1_str = h3.int_to_str(cell1)
    cell2_str = h3.int_to_str(cell2)
    min_res = min(h3.get_resolution(cell1_str), h3.get_resolution(cell2_str))
    for res in range(min_res, -1, -1):
        if h3.cell_to_parent(cell1_str, res) == h3.cell_to_parent(cell2_str, res):
            return h3.str_to_int(h3.cell_to_parent(cell1_str, res))
    return 0
    
def parent_check(node_cell: int, high_cell: int, high_res: int) -> bool:
    """Check if node is within high_cell region."""
    if high_cell == 0 or high_res < 0:
        return True
    if node_cell == 0:
        return False
    
    try:
        node_str = h3.int_to_str(node_cell)
        node_res = h3.get_resolution(node_str)
        if high_res > node_res:
            return False
        parent = h3.str_to_int(h3.cell_to_parent(node_str, high_res))
        return parent == high_cell
    except:
        print(f"Error node_cell = {node_cell}, high_cell = {high_cell}, high_res = {high_res}")
        return False

def compute_high_cell(source_edge: int, target_edge: int) -> HighCell:
    """Compute highest common H3 cell for source/target."""
    src_meta = edge_meta[source_edge]
    dst_meta = edge_meta[target_edge]
    
    src_cell = src_meta['incoming_cell']
    dst_cell = dst_meta['incoming_cell']
    src_res = src_meta['lca_res']
    dst_res = dst_meta['lca_res']

    
    # Get cells at their LCA resolutions
    src_cell = safe_cell_to_parent(src_cell, src_res)
    dst_cell = safe_cell_to_parent(dst_cell, dst_res)

    if src_cell == 0 or dst_cell == 0:
        return HighCell(0, -1)
        
    lca = find_lca(src_cell, dst_cell)
    if lca != 0:
        lca_str = h3.int_to_str(lca)
        res = h3.get_resolution(lca_str)
    else:
        res = -1
    return HighCell(lca, res)

    
def get_edge_cell(edge_id: int) -> int:
    """Get H3 cell for an edge."""
    return edge_meta[edge_id]['incoming_cell']

    
def get_edge_cost(edge_id: int) -> float:
    """Get traversal cost for an edge."""
    return edge_meta[edge_id]['cost']

## Algorithm 1: Classic Bidirectional Dijkstra

In [9]:
def query_classic(source_edge: int, target_edge: int) -> QueryResult:
    """Classic bidirectional Dijkstra with inside filtering only."""
    if source_edge == target_edge:
        return QueryResult(get_edge_cost(source_edge), [source_edge], True)
    
    inf = float('inf')
    dist_fwd = {source_edge: 0.0}
    dist_bwd = {target_edge: get_edge_cost(target_edge)}
    parent_fwd = {source_edge: source_edge}
    parent_bwd = {target_edge: target_edge}
    
    pq_fwd = [(0.0, source_edge)]
    pq_bwd = [(dist_bwd[target_edge], target_edge)]
    
    best = inf
    meeting = None
    
    while pq_fwd or pq_bwd:
        # Forward step
        if pq_fwd:
            d, u = heappop(pq_fwd)
            if d > dist_fwd.get(u, inf):
                pass  # stale
            elif d < best:
                for sc in fwd_adj.get(u, []):
                    if sc.inside != 1:
                        continue
                    v = sc.to_edge
                    nd = d + sc.cost
                    if nd < dist_fwd.get(v, inf):
                        dist_fwd[v] = nd
                        parent_fwd[v] = u
                        heappush(pq_fwd, (nd, v))
                        if v in dist_bwd:
                            total = nd + dist_bwd[v]
                            if total < best:
                                best = total
                                meeting = v
        
        # Backward step
        if pq_bwd:
            d, u = heappop(pq_bwd)
            if d > dist_bwd.get(u, inf):
                pass  # stale
            elif d < best:
                for sc in bwd_adj.get(u, []):
                    if sc.inside not in (-1, 0):
                        continue
                    prev = sc.from_edge
                    nd = d + sc.cost
                    if nd < dist_bwd.get(prev, inf):
                        dist_bwd[prev] = nd
                        parent_bwd[prev] = u
                        heappush(pq_bwd, (nd, prev))
                        if prev in dist_fwd:
                            total = dist_fwd[prev] + nd
                            if total < best:
                                best = total
                                meeting = prev
        
        # Early termination
        if pq_fwd and pq_bwd:
            if pq_fwd[0][0] >= best and pq_bwd[0][0] >= best:
                break
        elif not pq_fwd and not pq_bwd:
            break
    
    if meeting is None or best == inf:
        return QueryResult(-1, [], False)
    
    # Reconstruct path
    path = []
    curr = meeting
    while curr != source_edge:
        path.append(curr)
        curr = parent_fwd[curr]
    path.append(source_edge)
    path.reverse()
    
    curr = meeting
    while curr != target_edge:
        curr = parent_bwd[curr]
        path.append(curr)
    
    return QueryResult(best, path, True)

## Algorithm 2: Pruned Bidirectional Dijkstra

In [10]:
def query_pruned(source_edge: int, target_edge: int) -> QueryResult:
    """Pruned bidirectional Dijkstra with H3 parent_check."""
    if source_edge == target_edge:
        return QueryResult(get_edge_cost(source_edge), [source_edge], True)
    
    high = compute_high_cell(source_edge, target_edge)
    
    inf = float('inf')
    dist_fwd = {source_edge: 0.0}
    dist_bwd = {target_edge: get_edge_cost(target_edge)}
    parent_fwd = {source_edge: source_edge}
    parent_bwd = {target_edge: target_edge}
    
   # Calculate initial edge cells using edge's lca_res
    src_meta = edge_meta[source_edge]
    src_cell = safe_cell_to_parent(src_meta['incoming_cell'], src_meta['lca_res'])
    tgt_meta = edge_meta[target_edge]
    tgt_cell = safe_cell_to_parent(tgt_meta['incoming_cell'], tgt_meta['lca_res']) 
    
    # Heap entries: (distance, edge_id, cell)
    pq_fwd = [(0.0, source_edge, src_cell)]
    pq_bwd = [(dist_bwd[target_edge], target_edge, tgt_cell)]
    
    best = inf
    meeting = None
    min_arrival_fwd = inf
    min_arrival_bwd = inf
      
    while pq_fwd or pq_bwd:
        # Forward step
        if pq_fwd:
            d, u, u_cell = heappop(pq_fwd)      
            if u in dist_bwd:
                        min_arrival_fwd = min(dist_fwd[u], min_arrival_fwd)
                        min_arrival_bwd = min(dist_bwd[u], min_arrival_bwd)
                        total = d + dist_bwd[u]
                        if total <= best:
                            best = total
                            meeting = u
                            
            if d >= best:
                continue
            if d > dist_fwd.get(u, inf):
                continue  # stale
            
            # PRUNING: Check popped node's cell
            if not parent_check(u_cell, high.cell, high.res):
                min_arrival_fwd = min(dist_fwd[u], min_arrival_fwd)
                continue

            if u_cell==high.cell:
                min_arrival_fwd = min(dist_fwd[u], min_arrival_fwd)

            for sc in fwd_adj.get(u, []):
                if sc.inside != 1:
                    continue
                v = sc.to_edge
                nd = d + sc.cost
                if nd < dist_fwd.get(v, inf):
                    dist_fwd[v] = nd
                    parent_fwd[v] = u
                    heappush(pq_fwd, (nd, v, sc.cell))  # Use sc.cell directly
                    
        
        # Backward step
        if pq_bwd:
            d, u, u_cell = heappop(pq_bwd)
            if u in dist_fwd:
                min_arrival_fwd = min(dist_fwd[u], min_arrival_fwd)
                min_arrival_bwd = min(dist_bwd[u], min_arrival_bwd)
                total = dist_fwd[u] + d
                if total < best:
                    best = total
                    meeting = u
                    
            if d > dist_bwd.get(u, inf):
                continue  # stale
            if d >= best:
                continue
           
            
            # PRUNING: Check popped node's cell
            check = parent_check(u_cell, high.cell, high.res)
            
            if u_cell==high.cell or (not check):
                min_arrival_bwd = min(dist_bwd[u], min_arrival_bwd)
            
            # Check if at high_cell for lateral edges
            at_high_cell = (u_cell == high.cell)
            
            for sc in bwd_adj.get(u, []):
                if sc.inside == -1 and check :
                    pass
                elif sc.inside == 0 and (at_high_cell or (not check)):
                    pass
                elif sc.inside == -2 and (not check):
                    pass
                else:
                    continue
                
                prev = sc.from_edge
                nd = d + sc.cost
                if nd < dist_bwd.get(prev, inf):
                    dist_bwd[prev] = nd
                    parent_bwd[prev] = u
                    heappush(pq_bwd, (nd, prev, sc.cell))  # Use sc.cell directly
                   
        
        # Early termination
        
        # Check if both directions can improve
        if best < inf:
            bound_fwd = min_arrival_fwd
            bound_bwd = min_arrival_bwd
            if pq_fwd:
                bound_fwd = min(bound_fwd, pq_fwd[0][0])
            if pq_bwd:
                bound_bwd = min(bound_bwd, pq_bwd[0][0])
                
            fwd_good = pq_fwd and (pq_fwd[0][0] + bound_bwd < best)
            bwd_good = pq_bwd and (pq_bwd[0][0] + bound_fwd < best)
            if not fwd_good and not bwd_good:
                break

    
    if meeting is None or best == inf:
        return QueryResult(-1, [], False)
    
    # Reconstruct path
    path = []
    curr = meeting
    while curr != source_edge:
        path.append(curr)
        curr = parent_fwd[curr]
    path.append(source_edge)
    path.reverse()
    
    curr = meeting
    while curr != target_edge:
        curr = parent_bwd[curr]
        path.append(curr)
    
    return QueryResult(best, path, True)

# Algorthm mant-to-many

In [11]:
def query_multi(source_edges: list, target_edges: list,
                source_dists: list, target_dists: list) -> QueryResult:
    """Many-to-many bidirectional Dijkstra for KNN routing."""
    inf = float('inf')
    
    # Initialize forward from all sources
    dist_fwd, parent_fwd, pq_fwd = {}, {}, []
    for src, d in zip(source_edges, source_dists):
        if src in edge_meta:
            dist_fwd[src] = d
            parent_fwd[src] = src
            heappush(pq_fwd, (d, src))
    
    # Initialize backward from all targets
    dist_bwd, parent_bwd, pq_bwd = {}, {}, []
    for tgt, d in zip(target_edges, target_dists):
        if tgt in edge_meta:
            init_dist = get_edge_cost(tgt) + d
            dist_bwd[tgt] = init_dist
            parent_bwd[tgt] = tgt
            heappush(pq_bwd, (init_dist, tgt))
    
    best, meeting = inf, None
    
    while pq_fwd or pq_bwd:
        if pq_fwd:
            d, u = heappop(pq_fwd)
            if u in dist_bwd and d + dist_bwd[u] < best:
                best, meeting = d + dist_bwd[u], u
            if d >= best or d > dist_fwd.get(u, inf): continue
            for sc in fwd_adj.get(u, []):
                if sc.inside == 1:
                    nd = d + sc.cost
                    if nd < dist_fwd.get(sc.to_edge, inf):
                        dist_fwd[sc.to_edge], parent_fwd[sc.to_edge] = nd, u
                        heappush(pq_fwd, (nd, sc.to_edge))
        
        if pq_bwd:
            d, u = heappop(pq_bwd)
            if u in dist_fwd and dist_fwd[u] + d < best:
                best, meeting = dist_fwd[u] + d, u
            if d >= best or d > dist_bwd.get(u, inf): continue
            for sc in bwd_adj.get(u, []):
                if sc.inside in (-1, 0):
                    nd = d + sc.cost
                    if nd < dist_bwd.get(sc.from_edge, inf):
                        dist_bwd[sc.from_edge], parent_bwd[sc.from_edge] = nd, u
                        heappush(pq_bwd, (nd, sc.from_edge))
        
        if best < inf:
            if pq_fwd and pq_fwd[0][0] >= best: pq_fwd = []
            if pq_bwd and pq_bwd[0][0] >= best: pq_bwd = []
    
    if not meeting: return QueryResult(-1, [], False)
    
    path = []
    curr = meeting
    while parent_fwd[curr] != curr:
        path.append(curr); curr = parent_fwd[curr]
    path.append(curr); path.reverse()
    curr = meeting
    while parent_bwd[curr] != curr:
        curr = parent_bwd[curr]; path.append(curr)
    return QueryResult(best, path, True)

## Test & Compare Algorithms

In [12]:
# Get sample edges for testing
all_edges = list(edge_meta.keys())
print(f"Total edges: {len(all_edges)}")

# Pick a random sample
np.random.seed(42)
sample_edges = np.random.choice(all_edges, size=min(1000, len(all_edges)), replace=False)
print(f"Sample edges: {sample_edges[:5]}")

Total edges: 5900
Sample edges: [5747 1978 1760 3768 5402]


In [13]:
def compare_algorithms(source: int, target: int):
    """Compare classic vs pruned for a single query."""
    t0 = time.perf_counter()
    r_classic = query_classic(source, target)
    t_classic = (time.perf_counter() - t0) * 1000
    
    t0 = time.perf_counter()
    r_pruned = query_pruned(source, target)
    t_pruned = (time.perf_counter() - t0) * 1000
    
    match = abs(r_classic.distance - r_pruned.distance) < 0.001 if r_classic.reachable and r_pruned.reachable else r_classic.reachable == r_pruned.reachable
    
    return {
        'source': source,
        'target': target,
        'classic_dist': r_classic.distance,
        'pruned_dist': r_pruned.distance,
        'classic_ms': t_classic,
        'pruned_ms': t_pruned,
        'match': match,
        'speedup': t_classic / t_pruned if t_pruned > 0 else 0
    }

In [14]:
# Run comparison on sample pairs
results = []
for i, src in enumerate(sample_edges[:20]):
    for dst in sample_edges[:20]:
        if src != dst:
            r = compare_algorithms(src, dst)
            results.append(r)
            if not r['match']:
                print(f"MISMATCH: {src} -> {dst}: classic={r['classic_dist']:.3f}, pruned={r['pruned_dist']:.3f}")

results_df = pd.DataFrame(results)
print(f"\nTotal queries: {len(results_df)}")
print(f"Matches: {results_df['match'].sum()} / {len(results_df)}")
print(f"Avg speedup: {results_df['speedup'].mean():.2f}x")
results_df.head(10)


Total queries: 380
Matches: 380 / 380
Avg speedup: 1.67x


,source,target,classic_dist,pruned_dist,classic_ms,pruned_ms,match,speedup
0,5747,1978,-1.0,-1.0,18.744367,25.634755,True,0.731209
1,5747,1760,-1.0,-1.0,23.891540,99.336840,True,0.240510
2,5747,3768,-1.0,-1.0,22.261384,23.722625,True,0.938403
3,5747,5402,-1.0,-1.0,29.014230,27.245342,True,1.064924
4,5747,1362,-1.0,-1.0,22.158108,100.667624,True,0.220112
5,5747,2358,-1.0,-1.0,0.188449,0.259015,True,0.727560
6,5747,1181,-1.0,-1.0,30.513875,29.899094,True,1.020562
7,5747,5827,-1.0,-1.0,23.292115,112.558188,True,0.206934
8,5747,1047,-1.0,-1.0,32.891918,25.642204,True,1.282726
9,5747,626,-1.0,-1.0,19.632317,28.028190,True,0.700449


In [15]:
# Show any mismatches
mismatches = results_df[~results_df['match']]
if len(mismatches) > 0:
    print(f"\n{len(mismatches)} MISMATCHES FOUND:")
    print(mismatches)
else:
    print("\n✓ All results match!")


✓ All results match!


In [16]:
def query_pruned_trace(source_edge: int, target_edge: int):
    """Debug trace matching query_pruned exactly."""
    
    r_classic = query_classic(source_edge, target_edge)
    classic_path = set(r_classic.path) if r_classic.reachable else set()
    
    r_pruned = query_pruned(source_edge, target_edge)
    pruned_path = set(r_pruned.path) if r_pruned.reachable else set()
    
    high = compute_high_cell(source_edge, target_edge)
    print(f"High cell: {hex(high.cell)}, res: {high.res}")
    print(f"Classic path: {r_classic.path}, dist={r_classic.distance:.2f}")
    print(f"Pruned path: {r_pruned.path}, dist={r_pruned.distance:.2f}")
    
    trace = []
    inf = float('inf')
    dist_fwd = {source_edge: 0.0}
    dist_bwd = {target_edge: get_edge_cost(target_edge)}
    
    src_meta = edge_meta[source_edge]
    src_cell = safe_cell_to_parent(src_meta['incoming_cell'], src_meta['lca_res'])
    tgt_meta = edge_meta[target_edge]
    tgt_cell = safe_cell_to_parent(tgt_meta['incoming_cell'], tgt_meta['lca_res'])
    
    pq_fwd = [(0.0, source_edge, src_cell, None)]
    pq_bwd = [(dist_bwd[target_edge], target_edge, tgt_cell, None)]
    best = inf
    meeting = None
    
    while pq_fwd or pq_bwd:
        # Forward step
        if pq_fwd:
            d, u, u_cell, inside_val = heappop(pq_fwd)
            cell_res = h3.get_resolution(h3.int_to_str(u_cell)) if u_cell != 0 else -1
            
            skip = None
            met = False
            
            # Meeting check FIRST (before stale/d>=best)
            if u in dist_bwd:
                total = d + dist_bwd[u]
                if total <= best:
                    best = total
                    meeting = u
                    met = True
            
            if d >= best:
                skip = "d>=best"
            elif d > dist_fwd.get(u, inf):
                skip = "stale"
            elif not parent_check(u_cell, high.cell, high.res):
                skip = "parent_check"
            
            neighbors = len([sc for sc in fwd_adj.get(u, []) if sc.inside == 1])
            trace.append({
                'dir': 'FWD', 'edge': u, 'dist': round(d, 2), 'cell': hex(u_cell)[-8:],
                'res': cell_res, 'inside': inside_val, 'skip': skip or 'OK', 
                'met': met, 'neighbors': neighbors,
                'in_classic': u in classic_path, 'in_pruned': u in pruned_path
            })
            
            if skip:
                continue
            for sc in fwd_adj.get(u, []):
                if sc.inside == 1:
                    nd = d + sc.cost
                    if nd < dist_fwd.get(sc.to_edge, inf):
                        dist_fwd[sc.to_edge] = nd
                        heappush(pq_fwd, (nd, sc.to_edge, sc.cell, sc.inside))
        
        # Backward step
        if pq_bwd:
            d, u, u_cell, inside_val = heappop(pq_bwd)
            cell_res = h3.get_resolution(h3.int_to_str(u_cell)) if u_cell != 0 else -1
            
            skip = None
            met = False
            
            # Meeting check FIRST (before stale/d>=best)
            if u in dist_fwd:
                total = dist_fwd[u] + d
                if total < best:
                    best = total
                    meeting = u
                    met = True
            
            if d > dist_bwd.get(u, inf):
                skip = "stale"
            elif d >= best:
                skip = "d>=best"
            else:
                check = parent_check(u_cell, high.cell, high.res)
                if not check:
                    skip = "parent_check"
            
            at_high_cell = (u_cell == high.cell)
            check = parent_check(u_cell, high.cell, high.res)  # Need check for neighbor counting
            
            def allowed(sc):
                if sc.inside == -1 and check:
                    return True
                elif sc.inside == 0 and (at_high_cell or (not check)):
                    return True
                elif sc.inside == -2 and (not check):
                    return True
                return False
            neighbors = len([sc for sc in bwd_adj.get(u, []) if allowed(sc)])
            
            trace.append({
                'dir': 'BWD', 'edge': u, 'dist': round(d, 2), 'cell': hex(u_cell)[-8:],
                'res': cell_res, 'inside': inside_val, 'skip': skip or 'OK',
                'met': met, 'neighbors': neighbors,
                'in_classic': u in classic_path, 'in_pruned': u in pruned_path
            })
            
            if skip == "stale" or skip == "d>=best":
                continue
            
            for sc in bwd_adj.get(u, []):
                if sc.inside == -1 and check:
                    pass
                elif sc.inside == 0 and (at_high_cell or (not check)):
                    pass
                elif sc.inside == -2 and (not check):
                    pass
                else:
                    continue
                    
                nd = d + sc.cost
                if nd < dist_bwd.get(sc.from_edge, inf):
                    dist_bwd[sc.from_edge] = nd
                    heappush(pq_bwd, (nd, sc.from_edge, sc.cell, sc.inside))
        
        if len(trace) > 500:
            break
    
    print(f"Final: best={best:.2f}, meeting={meeting}")
    return pd.DataFrame(trace)

In [17]:
source_edge, target_edge = 3379, 346
trace_df = query_pruned_trace(source_edge, target_edge)

High cell: 0x882669a49bfffff, res: 8
Classic path: [3379, 3583, 3592, 3348, 1209, 4494, 325, 346], dist=175.87
Pruned path: [3379, 3583, 3592, 3348, 1209, 325, 346], dist=175.87
Final: best=175.87, meeting=1209


In [18]:
trace_df[trace_df["in_classic"]]

,dir,edge,dist,cell,res,inside,skip,met,neighbors,in_classic,in_pruned
0,FWD,3379,0.00,49ac9fff,11,NaN,OK,False,5,True,True
1,BWD,346,6.11,49b3ffff,9,NaN,OK,False,2,True,True
2,FWD,3583,1.46,49acffff,10,1.0,OK,False,12,True,True
3,BWD,325,15.07,0x0,-1,-1.0,parent_check,False,107,True,True
11,FWD,3592,3.57,49afffff,9,1.0,OK,False,11,True,True
30,FWD,3348,22.41,49bfffff,8,1.0,OK,True,17,True,True
38,FWD,1209,36.27,49ffffff,7,1.0,parent_check,True,17,True,True
39,FWD,1209,36.99,49ffffff,7,1.0,stale,False,17,True,True
43,FWD,1209,38.58,49ffffff,7,1.0,stale,False,17,True,True
95,BWD,4494,39.52,0x0,-1,0.0,parent_check,False,37,True,False


In [19]:
trace_df[trace_df["in_pruned"]]

,dir,edge,dist,cell,res,inside,skip,met,neighbors,in_classic,in_pruned
0,FWD,3379,0.00,49ac9fff,11,NaN,OK,False,5,True,True
1,BWD,346,6.11,49b3ffff,9,NaN,OK,False,2,True,True
2,FWD,3583,1.46,49acffff,10,1.0,OK,False,12,True,True
3,BWD,325,15.07,0x0,-1,-1.0,parent_check,False,107,True,True
11,FWD,3592,3.57,49afffff,9,1.0,OK,False,11,True,True
30,FWD,3348,22.41,49bfffff,8,1.0,OK,True,17,True,True
38,FWD,1209,36.27,49ffffff,7,1.0,parent_check,True,17,True,True
39,FWD,1209,36.99,49ffffff,7,1.0,stale,False,17,True,True
43,FWD,1209,38.58,49ffffff,7,1.0,stale,False,17,True,True
338,BWD,1209,139.60,49ffffff,7,-2.0,parent_check,False,66,True,True


In [20]:
# Classic path: [3379, 3583, 3592, 3348, 1209, ****, 4494, 325, 346], dist=175.87
# Pruned path: [3379, 3583, 3592, 3348, 1209, ****, 315, 346], dist=177.57

In [25]:
src, dst = sample_edges[10], sample_edges[676]
r_classic = query_classic(src, dst)
r_multi = query_multi([src], [dst], [0.0], [0.0])
print(f"Match: {abs(r_classic.distance - r_multi.distance) < 0.001}")

Match: True
